In [6]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import csv
import time
from requests.exceptions import ReadTimeout
import requests
import pandas as pd
from datetime import datetime

/Users/shixin/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [45]:

import requests
import base64

client_id ="1ec7e3d98ecb4de7994a3c40d22f6335"
client_secret = "4e933e325f8441e19592506f340759e0"

# client_id = "9f4661edc3c44ab98f58dad78e28e111"
# client_secret = "a9a2fc4e13514726847e9a44fddbea17"


auth_header = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()

response = requests.post(
    "https://accounts.spotify.com/api/token",
    headers={"Authorization": f"Basic {auth_header}"},
    data={"grant_type": "client_credentials"}
)

if response.status_code == 200:
    access_token = response.json().get("access_token")
    print("Access Token:", access_token)
else:
    print("Failed to get token:", response.json())


Access Token: BQAPOFdV1fZakEIHXHBDSLpTtF4z8uU-V5hwUSFQlmVt_57TduvxuAiUZ3xiOJCDK2mWMA-hLlAaPRwSRjAkClm1jvy6CExdZC7nhnlDDYD1Z9d93F8


In [46]:

# Access token for Spotify API
ACCESS_TOKEN = "BQAPOFdV1fZakEIHXHBDSLpTtF4z8uU-V5hwUSFQlmVt_57TduvxuAiUZ3xiOJCDK2mWMA-hLlAaPRwSRjAkClm1jvy6CExdZC7nhnlDDYD1Z9d93F8"

# Load podcast IDs from CSV
def load_podcast_ids_from_csv(filename="podcasts.csv"):
    """
    Loads podcast IDs from a CSV file.

    Args:
        filename (str): Name of the CSV file containing podcast data.

    Returns:
        list: A list of unique podcast IDs.
    """
    podcast_ids = set()
    try:
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                podcast_ids.add(row["ID"])  # Assumes the CSV has an "ID" column
        print(f"Loaded {len(podcast_ids)} podcast IDs from {filename}.")
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
    return list(podcast_ids)

# Get all episodes for a show
def get_episodes_for_show(show_id):
    """
    Fetch all episodes for a given show ID from Spotify API.

    Args:
        show_id (str): Spotify show ID.

    Returns:
        list: List of dictionaries containing episode details, and whether the rate limit was reached.
    """
    url = f"https://api.spotify.com/v1/shows/{show_id}/episodes?limit=50"
    headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}
    
    episodes = []
    while url:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            if "items" in data:
                for item in data["items"]:
                    if item is not None:  # Ensure the item is not None
                        episodes.append({
                            "id": item.get("id", ""),
                            "description": item.get("description", ""),
                            "duration_ms": item.get("duration_ms", 0),
                            "explicit": item.get("explicit", False),
                            "is_externally_hosted": item.get("is_externally_hosted", False),
                            "is_playable": item.get("is_playable", True),
                            "languages": ", ".join(item.get("languages", [])),
                            "name": item.get("name", ""),
                            "release_date": item.get("release_date", ""),
                            "release_date_precision": item.get("release_date_precision", ""),
                            "show_id": show_id,
                        })
            # Check if there is a next page
            url = data.get("next")
        elif response.status_code == 429:  # Rate limit reached
            print(f"Rate limit reached for Show ID: {show_id}. Stopping...")
            return episodes, True  # Indicate rate limit reached
        else:
            print(f"Request failed: {response.status_code}, Show ID: {show_id}")
            break
    
    return episodes, False  # Normal completion without hitting rate limit

# Save data to CSV
def save_to_csv(data, filename_prefix):
    """
    Saves the given data to a CSV file, appending the current date to the filename.

    Args:
        data (list): List of dictionaries to save.
        filename_prefix (str): Prefix for the filename.
    """
    date_str = datetime.now().strftime("%Y-%m-%d")  # Get the current date
    filename = f"{filename_prefix}_{date_str}.csv"  # Add date to filename
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False, encoding="utf-8")
    print(f"Data saved to {filename}")

# Load podcast show IDs
show_ids = load_podcast_ids_from_csv("podcasts.csv")

# Initialize variables
all_episodes = []
start_index = 10652   # Update this to resume from a specific index if needed

# Fetch episodes for all shows
for index, show_id in enumerate(show_ids, start=start_index):
    print(f"{index}. Fetching episodes for Show ID: {show_id}")
    try:
        episodes, rate_limit_reached = get_episodes_for_show(show_id)
        all_episodes.extend(episodes)

        if rate_limit_reached:  # Stop if rate limit is reached
            print(f"Stopped due to rate limit. Last processed index: {index}")
            save_to_csv(all_episodes, "all_episodes_data"+str(start_index))  # Save with date
            break

    except Exception as e:
        print(f"Error fetching Show ID {show_id}: {e}")

# Save remaining data if not already saved
if all_episodes and not rate_limit_reached:
    save_to_csv(all_episodes, "all_episodes_data"+str(start_index))
print("Processing complete.")


Loaded 11282 podcast IDs from podcasts.csv.
10652. Fetching episodes for Show ID: 4fRq8UFF7QAcea7xzqf5lg
10653. Fetching episodes for Show ID: 3EnGW5tFIjPMWvWFcoCL1g
10654. Fetching episodes for Show ID: 1WTeUfz1fkqFAZ2VaWIi5V
10655. Fetching episodes for Show ID: 3dJMGMnZdNtwYYT3NCqDhd
10656. Fetching episodes for Show ID: 1s4vovGkxJZI1JVtXiR0Ot
10657. Fetching episodes for Show ID: 57QzRljdRyvfyXblrBQQ3q
10658. Fetching episodes for Show ID: 5YoA2ciDCXfhQCOjEdd2Vk
10659. Fetching episodes for Show ID: 78BMCe48RhJE7W3ZA0PDu4
10660. Fetching episodes for Show ID: 6c19uhHBcGF9FLqD2TCcnJ
10661. Fetching episodes for Show ID: 5yvXAxab0Uj5ombb2U6F2U
10662. Fetching episodes for Show ID: 4fReACEwyBerw1SdWyeO1v
10663. Fetching episodes for Show ID: 6WUgeZRyQK1clQ9loJsk4j
10664. Fetching episodes for Show ID: 3Qn9YbK2gubV5vnxQ77uub
10665. Fetching episodes for Show ID: 6xwbpXMGtyutCFz0lg4dv7
10666. Fetching episodes for Show ID: 3WqEZXavqg3qstoLKwtllF
10667. Fetching episodes for Show ID: 4vE

10785. Fetching episodes for Show ID: 7x7wyfYs7fG88z2f3lKixq
10786. Fetching episodes for Show ID: 0akHL0OhXgxuF0K0N7cA5s
10787. Fetching episodes for Show ID: 7lAX0i1SOJfTb2HJrzGlBK
10788. Fetching episodes for Show ID: 5TUxj9LJJpArLU9C3XxGpT
10789. Fetching episodes for Show ID: 5bxXL5MdfdTyyDy4RBg3AK
10790. Fetching episodes for Show ID: 2bWjI6CbJvfHT4WjtGLjMO
10791. Fetching episodes for Show ID: 32ADB4Q1RuNcTHrJ6lQ5lX
10792. Fetching episodes for Show ID: 4nz32MWBscoelt1vEu2uzV
10793. Fetching episodes for Show ID: 1mffoIiAsKDVXhDEVlsuLF
10794. Fetching episodes for Show ID: 6v4PXYlNTwkNQ20lfhVu2w
10795. Fetching episodes for Show ID: 7rd3o1SPCSVIVKwCMOLkK9
10796. Fetching episodes for Show ID: 16T7BnJLoQgkkF9apyVoqB
10797. Fetching episodes for Show ID: 7rz1WlawSs9qB7eRIrOy0A
10798. Fetching episodes for Show ID: 4pPjIJ3sR1rKIOCwee48ZV
10799. Fetching episodes for Show ID: 4alur1jkJX7RsPCVVMDdEP
10800. Fetching episodes for Show ID: 61CL9Ba7hnPP53xsfKbSPs
10801. Fetching episodes

10918. Fetching episodes for Show ID: 2ICy53cWZCghqnsPTJgqOZ
10919. Fetching episodes for Show ID: 1ix4xnADzJYtNmqPlD577K
10920. Fetching episodes for Show ID: 3twhc87dJrDekc39c7ibQE
10921. Fetching episodes for Show ID: 1nmHGhfdkuJEJIczs2q8Kx
10922. Fetching episodes for Show ID: 2IbydzcCIa6O7LAsTu6sFP
10923. Fetching episodes for Show ID: 0f0pcN0L25YwLPhvigqtBi
10924. Fetching episodes for Show ID: 6wHaoxlfmHhziFkS45Z3Qo
10925. Fetching episodes for Show ID: 6MH5MDw1e3rHai6tDGxj5e
10926. Fetching episodes for Show ID: 2ISm4FnCv1wq3YoPXRLYOT
10927. Fetching episodes for Show ID: 1Gq4FyY0Y83QyS0DxBl5aZ
10928. Fetching episodes for Show ID: 1rreB4z2QPz4I8lmLjvjsk
10929. Fetching episodes for Show ID: 1evOS2WyUpfHSuwTF2zm1L
10930. Fetching episodes for Show ID: 13EJVHvRuyazOtXVjNbDuq
10931. Fetching episodes for Show ID: 620t1f6VDuSwlxamWlyNzJ
10932. Fetching episodes for Show ID: 47Jc1zDYaGrg9CHaepiI0B
10933. Fetching episodes for Show ID: 0HmiyOVfuiL5J5RcHrOQSt
10934. Fetching episodes

11053. Fetching episodes for Show ID: 4sSKxyS7Yt4MFGutvcuwsg
11054. Fetching episodes for Show ID: 1KQP3qxNBjyUfgJhNCcs8f
11055. Fetching episodes for Show ID: 6E1u3kxII5CbbFR4VObax4
11056. Fetching episodes for Show ID: 6FaGGin7x6uhF173tT9bYp
11057. Fetching episodes for Show ID: 4FFMoPyMjXqnMncSK2RprB
11058. Fetching episodes for Show ID: 4rvivHyyv7poHhZDgp29MX
11059. Fetching episodes for Show ID: 1JeYsvYZI4OxGTC9TJljLV
11060. Fetching episodes for Show ID: 4ZxQ59e0rbADnEJXAI84XL
11061. Fetching episodes for Show ID: 2JVuKBTJsJcehqfcRJBweC
11062. Fetching episodes for Show ID: 5IjSotMcXVMSUXAg26bo2Z
11063. Fetching episodes for Show ID: 6qU0TAEVzxvA2GZPU1cTFP
11064. Fetching episodes for Show ID: 6KHwEnVRZmS9tmxkyfqUMD
11065. Fetching episodes for Show ID: 7LaQ4VeTP4e8BoGn2lffR9
11066. Fetching episodes for Show ID: 44BcTpDWnfhcn02ADzs7iB
11067. Fetching episodes for Show ID: 2wAMrXcoi3rBMjFzlkCK4B
11068. Fetching episodes for Show ID: 6JnQnqxVQECXT3RqQWDkNy
11069. Fetching episodes

11188. Fetching episodes for Show ID: 55vG1j5hNc1R6J0rO4Yh1h
11189. Fetching episodes for Show ID: 5szMpHbHXnrb00PaLZ4lxH
11190. Fetching episodes for Show ID: 6kCMPmQxqL1gO0DNtqeyQl
11191. Fetching episodes for Show ID: 4lGckznduLTxp6E12PmBsn
11192. Fetching episodes for Show ID: 4lnLTgPtmOuVC3n0A0uzax
11193. Fetching episodes for Show ID: 18RPgkjKw4JQDBBvCR1eQL
11194. Fetching episodes for Show ID: 0AJ8h8gfKodFQV6qFxd2tx
11195. Fetching episodes for Show ID: 45TVqGnlT6GsueaZIndSL5
11196. Fetching episodes for Show ID: 5qT2HRxVCJ4FGD2pTlLQw1
11197. Fetching episodes for Show ID: 6kYtPbP9vLVg1xSPAmKdcp
11198. Fetching episodes for Show ID: 1eDeav0UNrBQOSnxSKOqyr
11199. Fetching episodes for Show ID: 5ZgmRfMfTC7cgbtWhnPDYm
11200. Fetching episodes for Show ID: 5zIApywiP9iWg89lCwNda3
11201. Fetching episodes for Show ID: 1x7P8Me67ymbBk6c9lGhQW
11202. Fetching episodes for Show ID: 0iPziZ80E671LdGkUNjtXl
11203. Fetching episodes for Show ID: 5pgIJCrniYyXeXBEh37TvD
11204. Fetching episodes

11322. Fetching episodes for Show ID: 1hygb4nGhNhlLn4pBnN00j
11323. Fetching episodes for Show ID: 3a34llttOShHkZBH7OWweT
11324. Fetching episodes for Show ID: 027WSUNFvTmAJVIHumllqK
11325. Fetching episodes for Show ID: 1l5XeUZ9BOLcWY1AH3zue7
Request failed: 404, Show ID: 1l5XeUZ9BOLcWY1AH3zue7
11326. Fetching episodes for Show ID: 7ARINaz7N2ji2Q97MLjnCJ
11327. Fetching episodes for Show ID: 5LUnGGXpNoeaH1K0tWdLiB
11328. Fetching episodes for Show ID: 2gbsYruAIXC0A4g4pRJiQt
11329. Fetching episodes for Show ID: 0SA1gMrkpukXtTOy2yR3ng
11330. Fetching episodes for Show ID: 4exvsUsrAvWnrap7fNcJC8
11331. Fetching episodes for Show ID: 3g30sZPY7l8ETx33IJGMTa
11332. Fetching episodes for Show ID: 2AoYk2xxbBfVjagnt4mwuV
11333. Fetching episodes for Show ID: 238kNlFMjH1WKIwA09HP9F
11334. Fetching episodes for Show ID: 0aZIlINsnXl0VbDx4eNkNP
11335. Fetching episodes for Show ID: 7iAVJ3zVTGv9omVDXrqlyG
11336. Fetching episodes for Show ID: 3EOIK9y54dK2R2OrpH9lPo
11337. Fetching episodes for Sho

In [58]:
import os
import pandas as pd


output_file = "episodes.csv"

csv_files = [f for f in os.listdir('.') if f.startswith("all_episodes_data") and f.endswith(".csv")]

if not csv_files:
    print("No files found with prefix 'all_episodes_data'. Exiting...")
    exit()

data_frames = []
for csv_file in csv_files:
    print(f"Processing {csv_file}...")
    try:
        df = pd.read_csv(csv_file)
        data_frames.append(df)
    except Exception as e:
        print(f"Error reading {csv_file}: {e}")

if data_frames:
    combined_df = pd.concat(data_frames, ignore_index=True)
    combined_df.to_csv(output_file, index=False)
    print(f"All files have been merged into {output_file}.")
else:
    print("No valid data frames to merge.")


Processing all_episodes_data1.csv...
Processing all_episodes_data2663_2024-11-28.csv...
Processing all_episodes_data2.csv...
Processing all_episodes_data5575_2024-11-30.csv...
Processing all_episodes_data4849_2024-11-30.csv...
Processing all_episodes_data7754_2024-12-02.csv...
Processing all_episodes_data4121_2024-11-29.csv...
Processing all_episodes_data9204_2024-12-03.csv...
Processing all_episodes_data3392_2024-11-29.csv...
Processing all_episodes_data1932_2024-11-28.csv...
Processing all_episodes_data8479_2024-12-02.csv...
Processing all_episodes_data.csv...
Processing all_episodes_data6301_2024-12-01.csv...
Processing all_episodes_data9933_2024-12-03.csv...
Processing all_episodes_data7027_2024-12-01.csv...
Processing all_episodes_data10652_2024-12-04.csv...
Processing all_episodes_data_2024-11-27.csv...
All files have been merged into episodes.csv.


In [159]:
df2 = combined_df[combined_df['languages'].str.contains('en', case=False, na=True)].drop_duplicates(subset=['id'])


In [165]:
combined_df['languages'].unique()

array([nan, 'en-GB', 'ur', 'en', 'en-US', 'en-CA', 'ar', 'it', 'es-MX',
       'pt', 'es-ES', 'gu', 'zh-CN', 'es', 'pt-BR', 'hu', 'ta', 'de-DE',
       'en-IE', 'ru', 'fr', 'zh-TW', 'en-AU', 'tr', 'vi', 'es-NI', 'ps',
       'ko-KO', 'ja', 'de', 'de-CH', 'en-UK', 'id', 'fr-FR', 'en-PH',
       'es-CR', 'el', 'nl', 'tl', 'sv', 'pt-PT', 'es-PE', 'fr-CA', 'pl'],
      dtype=object)

In [167]:
df3 = df2[(df2['description'].notnull()) & (df2['duration_ms'].notnull())]

In [171]:
df3.to_csv('episodes_V2.csv')